<a href="https://colab.research.google.com/github/AftabQuant/Deep-Learning/blob/main/Hyperparameter_%5BKeras_Tuner%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras-tuner --upgrade

In [2]:
import keras_tuner as kt
import tensorflow
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('diabetes.csv')

In [4]:
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [5]:
df.shape

(768, 9)

In [6]:
x = df.iloc[ : , : -1]
y = df['Outcome']

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import L2

In [8]:
ss = StandardScaler()
x = ss.fit_transform(x)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# **Keras Tuner for optimizers and loss function**

In [10]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = hp.Choice('Optimizers', values = ['adam', 'rmsprop', 'sgd'])
    loss = hp.Choice('Loss', values = ['binary_crossentropy', 'mean_squared_error'])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [11]:
tuner1 = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 5 ,
    directory='mydir',
    project_name = 'tuner1'
)

Reloading Tuner from mydir/tuner1/tuner0.json


In [12]:
tuner1.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [13]:
tuner1.get_best_hyperparameters()[0].values

{'Optimizers': 'adam', 'Loss': 'binary_crossentropy'}

# **Keras Tuner - [Num of Neurons]**

In [14]:
def build_model(hp):
    model = Sequential()
    units1 = hp.Int('units1', min_value = 8, max_value = 128)
    units2 = hp.Int('units2', min_value = 8, max_value = 128, step = 16)
    model.add(Dense(units=units1, activation='relu', input_dim=8))
    model.add(Dense(units=units2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
tuner2 = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name = 'tuner2'

)

Reloading Tuner from mydir/tuner2/tuner0.json


In [16]:
tuner2.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [17]:
tuner2.get_best_hyperparameters()[0].values

{'units1': 93, 'units2': 104}

# **Keras Tuner - [Num of Hidden layers]**

In [18]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [19]:
tuner3 = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name = 'tuner3'
)

Reloading Tuner from mydir/tuner3/tuner0.json


In [20]:
tuner3.search(x_train, y_train, epochs = 5, validation_data=(x_test, y_test))

In [21]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 5}

# **Hyperparameter - [Multiple Things]**

In [ ]:
def build_model(hp):
    model = Sequential()
    counter = 0

    # How many hidden layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10, step=1)):

        # For 1st layer
        if counter == 0:
            model.add(Dense(
                units=hp.Int('units' + str(i), min_value=8, max_value=128, step=4),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid', 'elu']),
                kernel_regularizer=L2(hp.Float(f'l2_{i}', min_value=1e-5, max_value=1e-2, sampling='LOG')),
                input_dim=8
            ))
        # For other layers
        else:
            model.add(Dense(
                units=hp.Int('units' + str(i), min_value=8, max_value=128, step=4),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid', 'elu']),
                kernel_regularizer=L2(hp.Float(f'l2_{i}', min_value=1e-5, max_value=1e-2, sampling='LOG'))
            ))
            dropout_rate = hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)
            model.add(Dropout(dropout_rate))

        counter += 1

        model.add(Dense(1, activation='sigmoid'))

        model.compile(
        optimizer=hp.Choice('optimizers', values=['adam', 'rmsprop', 'sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy']
        )

        return model

In [28]:
tuner_all = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name = 'alltuner1'
)

In [29]:
tuner_all.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 16s


In [30]:
tuner_all.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 96,
 'activation': 'relu',
 'l2_0': 0.0003748412695339138,
 'optimizers': 'rmsprop'}

In [31]:
model = tuner_all.get_best_models(num_models=1)[0]

In [32]:
model.fit(x_train, y_train, epochs=200, initial_epoch=5, validation_data=(x_test, y_test),verbose=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)
round(accuracy_score(y_test, y_pred)*100, 3)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


72.727

In [40]:
from sklearn.metrics import accuracy_score
y_train_pred = model.predict(x_train)
y_train_pred = (y_train_pred > 0.5)
round(accuracy_score(y_train, y_train_pred)*100, 3)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


83.876